In [16]:
import nibabel as nb
import numpy as np 
import scipy as sp 
import pandas as pd 
import glob
import os
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
#for session in sorted(glob.glob('/cbica/projects/spatiotemp_dev_plasticity/Inhibition/Alpraz/BIDS/*/*')):
 
    # define subid, sesid, and ses variables
   ## subid=session[65:-11]
   # sesid=session[-10:]
   # ses=sesid[-5:]
    subid="sub-010122"
    sesid="ses-001724"
    ses="01724"
    
    # get TR length and number
    img = nb.load('/cbica/projects/spatiotemp_dev_plasticity/Inhibition/Alpraz/fmriprep/fmriprep/{0}/{1}/func/{0}_{1}_task-emotionid_space-MNI152NLin6Asym_res-2_desc-preproc_bold.nii.gz'.format(subid,sesid))
    TR = img.header.get_zooms()[-1]
    coln = img.shape[-1]
    
    # create empty taskarray to save deconvolved HRF task signals to for neutral, angry, fear, sad, and happy conditions
    taskarray = np.empty(shape=(210,0))
    
    # read in each azid_subj${emotion}_3col.txt file
    dir = "/cbica/projects/spatiotemp_dev_plasticity/Inhibition/Alpraz/task_files/{0}/{1}".format(subid,sesid)
    for file in sorted(os.listdir(dir)):
        if file.startswith('{0}_azid_subj'.format(ses)):
            print(file)
            # real time for each timepoint 
            nTRs = np.linspace(0, (coln-1)*TR, coln)

            # initializes timespoint and correponing weight
            # weight is zeros for all time point initially
            hk = np.zeros([coln,2])
            hk[:,0] = nTRs
            dd =  pd.read_csv('/cbica/projects/spatiotemp_dev_plasticity/Inhibition/Alpraz/task_files/{0}/{1}/{2}'.format(subid,sesid,file),header=None,delimiter='  ')

            # get the task file and convert event to serial (from block)
            serialtime = []
            for j in range(0,len(dd)):
                serialtime.append(dd[0][j]) # add start of stimulus
                nn = np.int(np.round(dd[1][j]/TR)) # get number of timepoints between blocks
                if nn > 0: 
                    for i in range(1,nn+1):
                        serialtime.append(dd[0][j] + i*TR) # add timepoints between blocks if any

            # mark the stimulus time  
                for j in serialtime:
                    # indxxx = np.int(np.where(hk[:,0] == j)[0])
                    indxxy = np.where(hk[:,0] >= j)[0]
                    indxxx = np.int(indxxy[0])
                    hk[indxxx,1]=1 # each event timepoints is now 1 otherwise remains zerosent timepoints is now 1 otherwise remains zeros

            print(hk)
            # hrf
            from scipy.stats import gamma
            def hrf(times):
                """ Return values for HRF at given times """
                 # Gamma pdf for the peak
                peak_values = gamma.pdf(times, 6)
                 # Gamma pdf for the undershoot
                undershoot_values = gamma.pdf(times, 12)
                 # Combine them
                values = peak_values - 0.35 * undershoot_values
                 # Scale max to 0.6
                return values / np.max(values) * 0.6

            # compute HRF with the signal
            hrf_times = np.arange(0, 35, TR)
            hrf_signal = hrf(hrf_times)
            N=len(hrf_signal)-1
            tt=np.convolve(hk[:,1],hrf_signal)
            realt=tt[:-N] # this is what we need actually
            print(realt)
            
            # combine realt deconvolved signals for each task condition in one array
            taskarray = np.column_stack((taskarray, realt))
            
    # save taskarray to custom task regression confounds .tsv file
    print("saving file for {0} and {1}".format(subid,sesid))
    #np.savetxt(fname="/cbica/projects/spatiotemp_dev_plasticity/Inhibition/Alpraz/task_files/{0}/{1}/{0}_{1}_task-emotionid_desc-custom_timeseries.tsv".format(subid,sesid),X=taskarray,delimiter='  ')

01724_azid_subjangry_3col.txt
[[  0.   0.]
 [  3.   0.]
 [  6.   0.]
 [  9.   0.]
 [ 12.   0.]
 [ 15.   0.]
 [ 18.   0.]
 [ 21.   0.]
 [ 24.   0.]
 [ 27.   0.]
 [ 30.   0.]
 [ 33.   0.]
 [ 36.   1.]
 [ 39.   1.]
 [ 42.   1.]
 [ 45.   0.]
 [ 48.   0.]
 [ 51.   0.]
 [ 54.   1.]
 [ 57.   1.]
 [ 60.   1.]
 [ 63.   0.]
 [ 66.   0.]
 [ 69.   0.]
 [ 72.   0.]
 [ 75.   0.]
 [ 78.   0.]
 [ 81.   0.]
 [ 84.   0.]
 [ 87.   0.]
 [ 90.   1.]
 [ 93.   1.]
 [ 96.   1.]
 [ 99.   0.]
 [102.   0.]
 [105.   0.]
 [108.   0.]
 [111.   0.]
 [114.   0.]
 [117.   0.]
 [120.   0.]
 [123.   0.]
 [126.   0.]
 [129.   0.]
 [132.   0.]
 [135.   0.]
 [138.   0.]
 [141.   0.]
 [144.   0.]
 [147.   0.]
 [150.   0.]
 [153.   0.]
 [156.   0.]
 [159.   0.]
 [162.   0.]
 [165.   0.]
 [168.   0.]
 [171.   0.]
 [174.   1.]
 [177.   1.]
 [180.   1.]
 [183.   0.]
 [186.   0.]
 [189.   0.]
 [192.   0.]
 [195.   0.]
 [198.   0.]
 [201.   0.]
 [204.   0.]
 [207.   0.]
 [210.   0.]
 [213.   0.]
 [216.   0.]
 [219.   0.]
 [222.  

[[  0.   1.]
 [  3.   1.]
 [  6.   1.]
 [  9.   0.]
 [ 12.   1.]
 [ 15.   1.]
 [ 18.   1.]
 [ 21.   1.]
 [ 24.   1.]
 [ 27.   1.]
 [ 30.   0.]
 [ 33.   0.]
 [ 36.   0.]
 [ 39.   0.]
 [ 42.   0.]
 [ 45.   0.]
 [ 48.   0.]
 [ 51.   0.]
 [ 54.   0.]
 [ 57.   0.]
 [ 60.   0.]
 [ 63.   0.]
 [ 66.   0.]
 [ 69.   0.]
 [ 72.   0.]
 [ 75.   0.]
 [ 78.   0.]
 [ 81.   0.]
 [ 84.   0.]
 [ 87.   0.]
 [ 90.   0.]
 [ 93.   0.]
 [ 96.   0.]
 [ 99.   0.]
 [102.   0.]
 [105.   0.]
 [108.   0.]
 [111.   0.]
 [114.   0.]
 [117.   1.]
 [120.   1.]
 [123.   1.]
 [126.   0.]
 [129.   0.]
 [132.   0.]
 [135.   0.]
 [138.   0.]
 [141.   0.]
 [144.   0.]
 [147.   0.]
 [150.   0.]
 [153.   0.]
 [156.   0.]
 [159.   0.]
 [162.   0.]
 [165.   0.]
 [168.   0.]
 [171.   0.]
 [174.   0.]
 [177.   0.]
 [180.   0.]
 [183.   0.]
 [186.   0.]
 [189.   0.]
 [192.   1.]
 [195.   1.]
 [198.   1.]
 [201.   0.]
 [204.   0.]
 [207.   0.]
 [210.   1.]
 [213.   1.]
 [216.   1.]
 [219.   0.]
 [222.   0.]
 [225.   0.]
 [228.   0.]

In [32]:
subid
dd
len(dd)
dd[0][1]
nn
serialtime
indxxy
indxxx
TR

3.0

In [15]:
hk

array([[  0.,   1.],
       [  3.,   1.],
       [  6.,   1.],
       [  9.,   0.],
       [ 12.,   1.],
       [ 15.,   1.],
       [ 18.,   1.],
       [ 21.,   1.],
       [ 24.,   1.],
       [ 27.,   1.],
       [ 30.,   0.],
       [ 33.,   0.],
       [ 36.,   0.],
       [ 39.,   0.],
       [ 42.,   0.],
       [ 45.,   0.],
       [ 48.,   0.],
       [ 51.,   0.],
       [ 54.,   0.],
       [ 57.,   0.],
       [ 60.,   0.],
       [ 63.,   0.],
       [ 66.,   0.],
       [ 69.,   0.],
       [ 72.,   0.],
       [ 75.,   0.],
       [ 78.,   0.],
       [ 81.,   0.],
       [ 84.,   0.],
       [ 87.,   0.],
       [ 90.,   0.],
       [ 93.,   0.],
       [ 96.,   0.],
       [ 99.,   0.],
       [102.,   0.],
       [105.,   0.],
       [108.,   0.],
       [111.,   0.],
       [114.,   0.],
       [117.,   1.],
       [120.,   1.],
       [123.,   1.],
       [126.,   0.],
       [129.,   0.],
       [132.,   0.],
       [135.,   0.],
       [138.,   0.],
       [141.,

In [20]:
df = pd.DataFrame(realt)

In [23]:
df.to_csv("/cbica/projects/spatiotemp_dev_plasticity/Inhibition/Alpraz/task_files/test.tsv",index = False,header=False,sep=' ')